In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader

In [2]:
class my_dataloader(data.Dataset):
    def __init__(self, pathx, pathy):
        self.X = torch.from_numpy(np.load(pathx)).float()
        self.Y = torch.from_numpy(np.load(pathy)).long()
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [16]:
class Net(nn.Module):
    fin_x = 0
    fin_y = 0
    image_x = 36
    image_y = 41
    k_size = 3
    stride = 2
    pool_size = 2
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(15, 32, self.k_size, self.stride)
        self.pool1 = nn.MaxPool2d(self.pool_size, self.pool_size)
        self.conv2 = nn.Conv2d(32, 64, self.k_size, self.stride)
        self.pool2 = nn.MaxPool2d(self.pool_size, self.pool_size)
        tmpx = int((int((self.image_x-self.k_size)/self.stride)+1)/self.pool_size)
        tmpy = int((int((self.image_y-self.k_size)/self.stride)+1)/self.pool_size)
        self.fin_x = int((int((tmpx-self.k_size)/self.stride)+1)/self.pool_size)
        self.fin_y = int((int((tmpy-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fc1 = nn.Linear(64 * self.fin_x * self.fin_y, 2048)
#         self.fc1 = nn.Linear(64 * self.fin_x * self.fin_y, 2)
        self.fc2 = nn.Linear(2048, 2)
        self.sof = nn.LogSoftmax()
#         self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
#         print(x.shape)
        x = x.view(-1, 64 * self.fin_x * self.fin_y)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.sof(x)
        return x

net = Net()
net = net.cuda()

In [17]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [5]:
batch_size = 100
train_set = my_dataloader('test_x_0_100_rgb.npy', 'test_y_0_100_rgb.npy')
train_generator = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

In [18]:
for epoch in range(10):
    running_loss = 0.0
    count = 0
    for x,y in train_generator:
#         print(x.shape)
        optimizer.zero_grad()
        outputs = net(x.cuda())
        loss = criterion(outputs, y.cuda())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        count += 1
    print('[%d] loss: %.3f' %(epoch + 1, running_loss / count))

/media/atishya/New_Volume/books_and_papers/Iit/3rd_Year/Sem2/COL772/Asignment/Project/conv-emotion/venv/lib/python3.5/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1] loss: 0.701
[2] loss: 0.689
[3] loss: 0.685
[4] loss: 0.683
[5] loss: 0.682
[6] loss: 0.680
[7] loss: 0.678
[8] loss: 0.678
[9] loss: 0.677
[10] loss: 0.676


In [19]:
# Test_NN_X = torch.from_numpy(np.load('test_x_0_100_rgb.npy')).float()
# Test_NN_Y = torch.from_numpy(np.load('test_y_0_100_rgb.npy')).long()
# preds_Test = torch.argmax(model(Test_NN_X), 1)
preds_Test = torch.argmax(net(train_set.X.cuda()), 1)

/media/atishya/New_Volume/books_and_papers/Iit/3rd_Year/Sem2/COL772/Asignment/Project/conv-emotion/venv/lib/python3.5/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [20]:
# print("----------Test Accuracy-----------")
# print(accuracy_score(Test_NN_Y, preds_Test))
# print(confusion_matrix(Test_NN_Y, preds_Test))
# print(classification_report(Test_NN_Y, preds_Test))

In [21]:
print("----------Test Accuracy-----------")
print(accuracy_score(train_set.Y, preds_Test))
print(confusion_matrix(train_set.Y, preds_Test))
print(classification_report(train_set.Y, preds_Test))

----------Test Accuracy-----------
0.5942726231386025
[[4557  211]
 [3331  631]]
              precision    recall  f1-score   support

           0       0.58      0.96      0.72      4768
           1       0.75      0.16      0.26      3962

   micro avg       0.59      0.59      0.59      8730
   macro avg       0.66      0.56      0.49      8730
weighted avg       0.66      0.59      0.51      8730

